In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  


In [2]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sci_plt
import csv  
from pprint import pprint
from sklearn.preprocessing import StandardScaler
from operations.fetch_data import PriceHistory
from keys.all_keys import alpha_vantage_api_key

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)

API call to collect data

In [4]:
api_key = alpha_vantage_api_key
symbols = {
        'tech': ['AAPL', 'GOOGL', 'MSFT', 'AMZN'],
        'finance': ['JPM', 'BAC', 'WFC'],
        'etfs': ['SPY', 'QQQ', 'VTI']
    }

number_of_symbols = len(symbols)
path = 'C:/projects/Son_of_anton/data/stocks.csv'
if not pathlib.Path(path).exists():
    price_history = PriceHistory(api_key=api_key, symbols=symbols)
    all_symbols = price_history.symbols()
    print("All symbols:", all_symbols)
    # Build URL example
    url = price_history.build_URL('AAPL')
    print("URL:", url)
    
    
    # Build unified dataframe (will use class symbols if none provided)
    unified_df = price_history.build_df()
    
    
    unified_df.to_csv(path)
    
    # Get summary
    summary = price_history.get_data_summary()
    print("Data Summary:", summary)

In [5]:
df = pd.read_csv('C:\projects\Son_of_anton\data\stocks.csv')
df.head()


,Unnamed: 0,date,open,high,low,close,volume,symbol,daily_return,price_range,avg_price
0,0,1999-11-01,80.00,80.69,77.37,77.62,2487300,AAPL,NaN,3.32,78.560000
1,1,1999-11-02,78.00,81.69,77.31,80.25,3564600,AAPL,0.033883,4.38,79.750000
2,2,1999-11-03,81.62,83.25,81.00,81.50,2932700,AAPL,0.015576,2.25,81.916667
3,3,1999-11-04,82.06,85.37,80.62,83.62,3384700,AAPL,0.026012,4.75,83.203333
4,4,1999-11-05,84.62,88.37,84.00,88.31,3721500,AAPL,0.056087,4.37,86.893333


In [6]:
# grab the cols we need
price_df = df[['date', 'symbol', 'close']]


#pivot the df to make the symbols the headers
price_df = price_df.pivot(
    index= 'date',
    columns='symbol',
    values='close'
)

price_df


symbol,AAPL,AMZN,BAC,GOOGL,JPM,MSFT,QQQ,SPY,VTI,WFC
date,,,,,,,,,,
1999-11-01,77.62,69.13,64.87,NaN,83.56,92.37,130.80,135.5625,NaN,46.88
1999-11-02,80.25,66.44,64.25,NaN,83.69,92.56,130.90,134.5937,NaN,47.38
1999-11-03,81.50,65.81,63.00,NaN,82.44,92.00,133.50,135.5000,NaN,46.13
1999-11-04,83.62,63.06,63.50,NaN,84.12,91.75,135.00,136.5312,NaN,47.00
1999-11-05,88.31,64.94,65.06,NaN,86.25,91.56,136.40,137.8750,NaN,46.81
...,...,...,...,...,...,...,...,...,...,...
2025-06-05,200.63,207.91,44.38,168.21,261.95,467.68,524.79,593.0500,291.72,74.90
2025-06-06,203.92,213.57,44.97,173.68,265.73,470.38,529.92,599.1400,295.12,76.33
2025-06-09,201.45,216.98,44.87,176.09,266.74,472.75,530.70,599.6800,295.36,76.46


In [7]:
# calculate the log return
log_return = np.log(1+price_df.pct_change())
#print('log_return:', log_return)

#calculating the number of symbols
number_of_symbols = len(df['symbol'].unique())

# weights (randomly assigned)
random_weights = np.array(np.random.random(number_of_symbols))
print('random_weights:', random_weights)

#rebalance weights (must be equals to 1)
rebalanced_weights = random_weights/np.sum(random_weights)
print('rebalanced_weights:', rebalanced_weights)

#calc expected Annualized returns (multipiled with 252 for annualization)
exp_returns =  np.sum((log_return.mean()*rebalanced_weights)*252)
print('expected returns:', exp_returns)

#calc expected volality annualized
exp_volt = np.sqrt(np.dot(rebalanced_weights.T,
                          np.dot(log_return.cov()*252,
                                 rebalanced_weights)))
print('exp_volt:', exp_volt)

#cal the sharp ratio
risk_free_rate = 0.0398 # us tresury yield 2025
print('r_f:', risk_free_rate)

sharp_ratio = (exp_returns-risk_free_rate)/exp_volt 
print('sharp_ratio:', sharp_ratio)




random_weights: [0.59279969 0.27587917 0.82079072 0.27161415 0.65134767 0.65404601
 0.62947766 0.82919264 0.31588276 0.68431726]
rebalanced_weights: [0.10353951 0.04818557 0.14336085 0.04744064 0.11376561 0.11423691
 0.10994575 0.14482835 0.05517268 0.11952414]
expected returns: 0.036487192818353824
exp_volt: 0.2653562929624491
r_f: 0.0398
sharp_ratio: -0.012484373913510231


Monte Carlo Simulation for Optimization

In [11]:
num_of_portfolios = 10000

# Weight array of numpy zeros
weights_arr = np.zeros((num_of_portfolios, number_of_symbols))

# return array
return_arr = np.zeros(num_of_portfolios)

#voletality array
vol_arr = np.zeros(num_of_portfolios)

# sharp ratio array
sharp_ratio_arr = np.zeros(num_of_portfolios)



In [16]:
# Portfolio simulation loop
for index in range(num_of_portfolios):
    # calc the weights
    weights = np.array(np.random.random(number_of_symbols))
    weights = weights/np.sum(weights)

    # add the weights to the weights array
    weights_arr[index, :] = weights

    # calc the expected log returns
    return_arr[index] = np.sum((log_return.mean() * weights) * 252)

    # calc the volatility and add to the vol_arr
    vol_arr[index] = np.sqrt(
        np.dot(weights.T, 
               np.dot(log_return.cov() * 252, weights))
    )

    # sharp ratio - FIXED: use vol_arr[index] instead of vol_arr
    # Also add check to avoid division by zero
    if vol_arr[index] != 0:
        sharp_ratio_arr[index] = (return_arr[index] - risk_free_rate) / vol_arr[index]
    else:
        sharp_ratio_arr[index] = 0  # or np.nan if you prefer

# combine them all together
simulation_data = [return_arr, vol_arr, sharp_ratio_arr, weights_arr]

# new df from the master array
simulation_df = pd.DataFrame(data=simulation_data).T

# add the col names
simulation_df.columns = [
    'Returns',
    'Volatility',
    'Sharpe Ratio',  
    'Portfolio Weights'
]

# making sure the datatype is correct; don't want floats to be integers
simulation_df = simulation_df.infer_objects()



In [17]:
print('')
print('=' * 250)
print('Simulation Results')  
print('-' * 250)
print(simulation_df.head())
print('-' * 250)


Simulation Results
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    Returns  Volatility  Sharpe Ratio                                                                                                                                                                                                      Portfolio Weights
0  0.047153    0.275861      0.026656  [0.1238518463956245, 0.15607784566498986, 0.030305152878742393, 0.03171418881862323, 0.16072075459796228, 0.19853259973121018, 0.021610053143693263, 0.19920871733397924, 0.049601775109770395, 0.028377066325404463]
1  0.035035    0.274809     -0.017339     [0.02455695784203381, 0.16833099341678867, 0.14688018482401344, 0.07913751567284774, 0.015130751484176289, 0.06581543359904032, 0.06604965081537494, 0.16418558616555137, 0.171841221